<a href="https://colab.research.google.com/github/TartuNLP/grammar-worker/blob/gec-and-spell/GEC_and_spell_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/TartuNLP/grammar-worker.git
%cd grammar-worker
! git checkout gec-and-spell
! pip install -r requirements.txt
! python -c "import nltk; nltk.download(\"punkt\")"

Cloning into 'grammar-worker'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 226 (delta 115), reused 149 (delta 59), pack-reused 0
Receiving objects: 100% (226/226), 54.56 KiB | 7.79 MiB/s, done.
Resolving deltas: 100% (115/115), done.
/content/grammar-worker
Branch 'gec-and-spell' set up to track remote branch 'gec-and-spell' from 'origin'.
Switched to a new branch 'gec-and-spell'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/TartuNLP/fairseq.git (to revision mtee-0.1.0) to /tmp/pip-req-build-flaion02
  Running command git clone --filter=blob:none --quiet https://github.com/TartuNLP/fairseq.git /tmp/pip-req-build-flaion02
  Running command git checkout -q 1a6f364b8af6e746dd1fc623c8cf670a0be5b696
  Resolved https://github.com/TartuNLP/fairseq.git to commit 1a6f364b8af6e746dd1fc623c8cf670a0be5b696
 

In [2]:
# kui mudelid kusagil mujal, saab ära vahetada
! git lfs install
! cd models && git clone https://huggingface.co/agnesluhtaru/GEC-noisy-nmt-ut
! cd models && git clone https://huggingface.co/agnesluhtaru/GEC-synthetic-pretrain-ut-ft
! mkdir models/spellmodels
! cd models/spellmodels && wget https://minitorn.tlu.ee/keelemudelid/etnc19_reference_corpus_model_6000000_lines.bin
! cd models/spellmodels && wget https://minitorn.tlu.ee/keelemudelid/etnc19_web_2019.bin
! cd models/spellmodels && wget https://minitorn.tlu.ee/keelemudelid/etnc19_reference_corpus_6000000_web_2019_600000.bin


Updated git hooks.
Git LFS initialized.
Cloning into 'GEC-noisy-nmt-ut'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), 63.25 KiB | 5.75 MiB/s, done.
Cloning into 'GEC-synthetic-pretrain-ut-ft'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), 63.46 KiB | 5.29 MiB/s, done.
Filtering content: 100% (4/4), 828.89 MiB | 50.28 MiB/s, done.
mkdir: cannot create directory ‘models/spellmodels’: File exists
--2023-03-13 09:48:47--  https://minitorn.tlu.ee/keelemudelid/etnc19_reference_corpus_model_6000000_lines.bin
Resolving minitorn.tlu.ee (minitorn.tlu.ee)... 193.40.238.40
Connecting to minitorn.tlu.ee (minitorn.tlu.ee)|193.40.238.40

In [3]:
from pprint import pprint
from dataclasses import asdict
from gec_worker import GEC, read_model_config
from gec_worker import spelling
from gec_worker.dataclasses import Response, Request
from gec_worker import multiple_corrections

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# Load grammatical error correction model
!ls models
# model_config = read_model_config('models/GEC-synthetic-pretrain-ut-ft-config.yaml')
model_config = read_model_config('models/GEC-noisy-nmt-ut-config.yaml')
gec = GEC(model_config)

GEC-noisy-nmt-ut	      GEC-synthetic-pretrain-ut-ft-config.yaml
GEC-noisy-nmt-ut-config.yaml  README.md
GEC-synthetic-pretrain-ut-ft  spellmodels


In [5]:
# Load spelling model, takes time 
speller = spelling.Spelling("etnc19_reference_corpus_6000000_web_2019_600000.bin")

In [6]:
# Make model list and add models
model_list = multiple_corrections.MultipleCorrections()
model_list.add_corrector(speller)
model_list.add_corrector(gec)

In [7]:
# Input data
source_text="Ükss väega vikase lause olema see"
request = Request(text=source_text, language='et')

In [8]:
# Only speller 

response = speller.process_request(request)
pprint(asdict(response))
response.corrected_text

{'corrected_text': 'Üks väega vigase lause olema see',
 'corrections': [{'replacements': [{'value': 'Üks'}],
                  'span': {'end': 4, 'start': 0, 'value': 'Ükss'}},
                 {'replacements': [{'value': 'vigase'}],
                  'span': {'end': 17, 'start': 11, 'value': 'vikase'}}],
 'original_text': 'Ükss väega vikase lause olema see',
 'status': 'OK',
 'status_code': 200}


'Üks väega vigase lause olema see'

In [9]:
# Only gec

response = gec.process_request(request)
pprint(asdict(response))
response.corrected_text

{'corrected_text': 'Üks vägeva vikase lause on see.',
 'corrections': [{'replacements': [{'value': 'Üks vägeva'}],
                  'span': {'end': 10, 'start': 0, 'value': 'Ükss väega'}},
                 {'replacements': [{'value': 'on see.'}],
                  'span': {'end': 33, 'start': 24, 'value': 'olema see'}}],
 'original_text': 'Ükss väega vikase lause olema see',
 'status': 'OK',
 'status_code': 200}


'Üks vägeva vikase lause on see.'

In [10]:
# Both
response = model_list.process_request(request)
pprint(asdict(response))
response.corrected_text

{'corrected_text': 'Üks väga vigane lause on see.',
 'corrections': [{'replacements': [{'value': 'Üks väga vigane'}],
                  'span': {'end': 17,
                           'start': 0,
                           'value': 'Ükss väega vikase'}},
                 {'replacements': [{'value': 'on see.'}],
                  'span': {'end': 33, 'start': 24, 'value': 'olema see'}}],
 'original_text': 'Ükss väega vikase lause olema see',
 'status': 'OK',
 'status_code': 200}


'Üks väga vigane lause on see.'

In [24]:
# Other GEC model

model_config = read_model_config('models/GEC-synthetic-pretrain-ut-ft-config.yaml')
# model_config = read_model_config('models/GEC-noisy-nmt-ut-config.yaml')
gec = GEC(model_config)

In [25]:
source_text_2="Ükss väega vikase lausele olema se siia"
request_2 = Request(text=source_text_2, language='et')

response = gec.process_request(request_2)
pprint(asdict(response))
response.corrected_text

{'corrected_text': 'Üks väga vigane lause on see siin.',
 'corrections': [{'replacements': [{'value': 'Üks väga vigane lause on see '
                                             'siin.'}],
                  'span': {'end': 39,
                           'start': 0,
                           'value': 'Ükss väega vikase lausele olema se '
                                    'siia'}}],
 'original_text': 'Ükss väega vikase lausele olema se siia',
 'status': 'OK',
 'status_code': 200}


'Üks väga vigane lause on see siin.'